# SQL Join Query Engine
In this tutorial, we show you how to use our SQLJoinQueryEngine.

This query engine allows you to combine insights from your structured tables with your unstructured data.
It first decides whether to query your structured tables for insights.
Once it does, it can then infer a corresponding query to the vector store in order to fetch corresponding documents.

### Setup

In [7]:
# NOTE: This is ONLY necessary in jupyter notebook.
# Details: Jupyter runs an event-loop behind the scenes. 
#          This results in nested event-loops when we start an event-loop to make async queries.
#          This is normally not allowed, we use nest_asyncio to allow it for convenience.  
import nest_asyncio
nest_asyncio.apply()

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [8]:
from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    ServiceContext,
    StorageContext,
    SQLStructStoreIndex, 
    SQLDatabase,
    WikipediaReader
)

### Create Common Objects

This includes a `ServiceContext` object containing abstractions such as the LLM and chunk size.
This also includes a `StorageContext` object containing our vector store abstractions.

In [9]:
# # define pinecone index 
# import pinecone
# import os

# api_key = os.environ['PINECONE_API_KEY']
# pinecone.init(api_key=api_key, environment="us-west1-gcp")

# # dimensions are for text-embedding-ada-002
# # pinecone.create_index("quickstart", dimension=1536, metric="euclidean", pod_type="p1")
# pinecone_index = pinecone.Index("quickstart")

In [10]:
# # OPTIONAL: delete all
# pinecone_index.delete(deleteAll=True)

In [31]:
from llama_index.node_parser.simple import SimpleNodeParser
from llama_index import ServiceContext, LLMPredictor
from llama_index.storage import StorageContext
from llama_index.vector_stores import PineconeVectorStore
from llama_index.langchain_helpers.text_splitter import TokenTextSplitter
from langchain.chat_models import ChatOpenAI

# define node parser and LLM
chunk_size = 1024
llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-4", streaming=True))
service_context = ServiceContext.from_defaults(chunk_size=chunk_size, llm_predictor=llm_predictor)
text_splitter = TokenTextSplitter(chunk_size=chunk_size)
node_parser = SimpleNodeParser(text_splitter=text_splitter)

# # define pinecone vector index
# vector_store = PineconeVectorStore(pinecone_index=pinecone_index, namespace='wiki_cities')
# storage_context = StorageContext.from_defaults(vector_store=vector_store)
# vector_index = VectorStoreIndex([], storage_context=storage_context)


### Create Database Schema + Test Data

Here we introduce a toy scenario where there are 100 tables (too big to fit into the prompt)

In [12]:
from sqlalchemy import create_engine, MetaData, Table, Column, String, Integer, select, column

In [13]:
engine = create_engine("sqlite:///:memory:", future=True)
metadata_obj = MetaData()

In [14]:
# create city SQL table
table_name = "city_stats"
city_stats_table = Table(
    table_name,
    metadata_obj,
    Column("city_name", String(16), primary_key=True),
    Column("population", Integer),
    Column("country", String(16), nullable=False),
)

metadata_obj.create_all(engine)

In [15]:
# print tables
metadata_obj.tables.keys()

dict_keys(['city_stats'])

We introduce some test data into the `city_stats` table

In [16]:
from sqlalchemy import insert
rows = [
    {"city_name": "Toronto", "population": 2930000, "country": "Canada"},
    {"city_name": "Tokyo", "population": 13960000, "country": "Japan"},
    {"city_name": "Berlin", "population": 3645000, "country": "Germany"},
]
for row in rows:
    stmt = insert(city_stats_table).values(**row)
    with engine.connect() as connection:
        cursor = connection.execute(stmt)
        connection.commit()

In [17]:
with engine.connect() as connection:
    cursor = connection.exec_driver_sql("SELECT * FROM city_stats")
    print(cursor.fetchall())

[('Toronto', 2930000, 'Canada'), ('Tokyo', 13960000, 'Japan'), ('Berlin', 3645000, 'Germany')]


### Load Data

We first show how to convert a Document into a set of Nodes, and insert into a DocumentStore.

In [12]:
# install wikipedia python package
!pip install wikipedia


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [18]:
cities = ['Toronto', 'Berlin', 'Tokyo']
wiki_docs = WikipediaReader().load_data(pages=cities)

### Build SQL Index

In [19]:
sql_database = SQLDatabase(engine, include_tables=["city_stats"])

In [20]:
sql_index = SQLStructStoreIndex.from_documents(
    [], 
    sql_database=sql_database, 
    table_name="city_stats",
)

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 0 tokens
> [build_index_from_nodes] Total embedding token usage: 0 tokens
> [build_index_from_nodes] Total embedding token usage: 0 tokens


### Build Vector Index

In [21]:
# Insert documents into vector index
# Each document has metadata of the city attached

vector_indices = {}
vector_query_engines = {}

for city, wiki_doc in zip(cities, wiki_docs):
    vector_index = VectorStoreIndex.from_documents([wiki_doc])
    query_engine = vector_index.as_query_engine(similarity_top_k=2)
    vector_indices[city] = vector_index
    vector_query_engines[city] = query_engine

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 17064 tokens
> [build_index_from_nodes] Total embedding token usage: 17064 tokens
> [build_index_from_nodes] Total embedding token usage: 17064 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 17943 tokens
> [build_index_from_nodes] Total embedding token usage: 17943 tokens
> [build_index_from_nodes] Total embedding token usage: 17943 tokens
INFO:llama_index.token_counter.token_counter:> [buil

### Define Query Engines, Set as Tools

In [25]:
from llama_index.query_engine import SQLJoinQueryEngine, RetrieverQueryEngine
from llama_index.tools.query_engine import QueryEngineTool
from llama_index.tools import ToolMetadata
from llama_index.indices.vector_store import VectorIndexAutoRetriever
from llama_index.query_engine import SubQuestionQueryEngine

In [26]:
sql_query_engine = sql_index.as_query_engine(synthesize_response=True)

In [27]:
from llama_index.query_engine import SubQuestionQueryEngine 

query_engine_tools = []
for city in cities:
    query_engine = vector_query_engines[city]
    
    query_engine_tool = QueryEngineTool(
        query_engine=query_engine, 
        metadata=ToolMetadata(name=city, description=f"Provides information about {city}")
    )
    query_engine_tools.append(query_engine_tool)
        

s_engine = SubQuestionQueryEngine.from_defaults(query_engine_tools=query_engine_tools)

# from llama_index.indices.vector_store.retrievers import VectorIndexAutoRetriever
# from llama_index.vector_stores.types import MetadataInfo, VectorStoreInfo
# from llama_index.query_engine.retriever_query_engine import RetrieverQueryEngine


# vector_store_info = VectorStoreInfo(
#     content_info='articles about different cities',
#     metadata_info=[
#         MetadataInfo(
#             name='title', 
#             type='str', 
#             description='The name of the city'),
#     ]
# )
# vector_auto_retriever = VectorIndexAutoRetriever(vector_index, vector_store_info=vector_store_info)

# retriever_query_engine = RetrieverQueryEngine.from_args(
#     vector_auto_retriever, service_context=service_context
# )


In [28]:
sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    description=(
        'Useful for translating a natural language query into a SQL query over a table containing: '
        'city_stats, containing the population/country of each city'
    )
)
s_engine_tool = QueryEngineTool.from_defaults(
    query_engine=s_engine,
    description=f'Useful for answering semantic questions about different cities',
)

### Define SQLJoinQueryEngine

In [32]:
query_engine = SQLJoinQueryEngine(
    sql_tool,
    s_engine_tool,
    service_context=service_context
)

In [33]:
response = query_engine.query('Tell me about the arts and culture of the city with the highest population')

Querying SQL database: Useful for translating a natural language query into a SQL query over a table containing city_stats, containing the population/country of each city
INFO:llama_index.query_engine.sql_join_query_engine:> Querying SQL database: Useful for translating a natural language query into a SQL query over a table containing city_stats, containing the population/country of each city
> Querying SQL database: Useful for translating a natural language query into a SQL query over a table containing city_stats, containing the population/country of each city
> Querying SQL database: Useful for translating a natural language query into a SQL query over a table containing city_stats, containing the population/country of each city
INFO:llama_index.indices.struct_store.sql_query:> Table desc str: Schema of table city_stats:
Table 'city_stats' has columns: city_name (VARCHAR(16)), population (INTEGER), country (VARCHAR(16)) and foreign keys: .

> Table desc str: Schema of table city_sta

In [24]:
print(str(response))

Tokyo, the city with the highest population of 13.96 million people, boasts a vibrant arts and culture scene. It offers a diverse range of art forms, from traditional Japanese art such as calligraphy and woodblock prints to modern art galleries and museums. Some notable art galleries and museums in Tokyo include the Nezu Museum in Aoyama, the National Diet Library, National Archives, and the National Museum of Modern Art near the Imperial Palace. The city also hosts various festivals and events throughout the year that celebrate its culture and art, such as the Sannō at Hie Shrine, the Sanja at Asakusa Shrine, and the biennial Kanda Festivals.


In [23]:
response = query_engine.query('Tell me about the history of Berlin')

Querying vector database: Useful for answering semantic questions about different cities
Vector DB response: Berlin's history dates back to the early 13th century when it was founded as a small settlement. In 1618, the Margraviate of Brandenburg entered into a personal union with the Duchy of Prussia, and in 1701, they formed the Kingdom of Prussia with Berlin as its capital. The city grew and merged with neighboring cities, becoming a center of the Enlightenment under the rule of Frederick the Great in the 18th century.

The Industrial Revolution in the 19th century transformed Berlin, expanding its economy, population, and infrastructure. In 1871, it became the capital of the newly founded German Empire. The early 20th century saw Berlin as a hub for the German Expressionist movement and a major world capital known for its contributions to science, technology, arts, and other fields.

In 1933, Adolf Hitler and the Nazi Party came to power, leading to a decline in Berlin's Jewish comm

In [24]:
print(str(response))

Berlin's history dates back to the early 13th century when it was founded as a small settlement. In 1618, the Margraviate of Brandenburg entered into a personal union with the Duchy of Prussia, and in 1701, they formed the Kingdom of Prussia with Berlin as its capital. The city grew and merged with neighboring cities, becoming a center of the Enlightenment under the rule of Frederick the Great in the 18th century.

The Industrial Revolution in the 19th century transformed Berlin, expanding its economy, population, and infrastructure. In 1871, it became the capital of the newly founded German Empire. The early 20th century saw Berlin as a hub for the German Expressionist movement and a major world capital known for its contributions to science, technology, arts, and other fields.

In 1933, Adolf Hitler and the Nazi Party came to power, leading to a decline in Berlin's Jewish community and the city's involvement in World War II. After the war, Berlin was divided into East and West Berlin

In [27]:
response = query_engine.query('Can you give me the country corresponding to each city?')

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.


Querying SQL database: Useful for translating a natural language query into a SQL query over a table containing: city_stats, containing the population/country of each city
SQL query: SELECT city_name, country FROM city_stats;
SQL response:  Toronto is in Canada, Tokyo is in Japan, and Berlin is in Germany.
Transformed query given SQL response: None


In [28]:
print(str(response))

 Toronto is in Canada, Tokyo is in Japan, and Berlin is in Germany.
